# 단어 수준 텍스트 생성

이 노트북은 아래 문서에 실려 있는 예제를 실습하면서 작성한 것입니다.

* [How to Develop a Word-Level Neural Language Model and Use it to Generate Text](https://machinelearningmastery.com/how-to-develop-a-word-level-neural-language-model-in-keras/) - Machine Learning Mastery

## 데이터 준비

* [Download The Republic by Plato (republic.txt)](http://www.gutenberg.org/cache/epub/1497/pg1497.txt)

### 텍스트 읽기

In [1]:
def load_doc(filename):
    with open(filename, mode='r', encoding='utf-8') as f:
        text = f.read()
        
    return text

in_filename = 'republic_clean.txt'
doc = load_doc(in_filename)
print(doc[:200])

﻿BOOK I.


I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in wh


### 텍스트 정제

In [2]:
import string

def clean_doc(doc):
    doc = doc.replace('--', ' ')
    tokens = doc.split()
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word.lower() for word in tokens]
    return tokens

tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['i', 'i', 'went', 'down', 'yesterday', 'to', 'the', 'piraeus', 'with', 'glaucon', 'the', 'son', 'of', 'ariston', 'that', 'i', 'might', 'offer', 'up', 'my', 'prayers', 'to', 'the', 'goddess', 'bendis', 'the', 'thracian', 'artemis', 'and', 'also', 'because', 'i', 'wanted', 'to', 'see', 'in', 'what', 'manner', 'they', 'would', 'celebrate', 'the', 'festival', 'which', 'was', 'a', 'new', 'thing', 'i', 'was', 'delighted', 'with', 'the', 'procession', 'of', 'the', 'inhabitants', 'but', 'that', 'of', 'the', 'thracians', 'was', 'equally', 'if', 'not', 'more', 'beautiful', 'when', 'we', 'had', 'finished', 'our', 'prayers', 'and', 'viewed', 'the', 'spectacle', 'we', 'turned', 'in', 'the', 'direction', 'of', 'the', 'city', 'and', 'at', 'that', 'instant', 'polemarchus', 'the', 'son', 'of', 'cephalus', 'chanced', 'to', 'catch', 'sight', 'of', 'us', 'from', 'a', 'distance', 'as', 'we', 'were', 'starting', 'on', 'our', 'way', 'home', 'and', 'told', 'his', 'servant', 'to', 'run', 'and', 'bid', 'us', '

### 정제된 텍스트 저장

In [3]:
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 117290


In [4]:
def save_doc(lines, filename):
    data = '\n'.join(lines)
    with open(filename, mode='w', encoding='utf-8') as f:
        f.write(data)
        
out_filename = 'republic_sequences.txt'
save_doc(sequences, out_filename)

## 언어 모델 훈련

### 훈련 데이터 읽기

In [5]:
def load_doc(filename):
    with open(filename, mode='r', encoding='utf-8') as f:
        text = f.read()
        
    return text

in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
print(lines[:2])

['i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted', 'i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted with']


### 훈련 데이터 인코딩

In [6]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [10]:
from numpy import array
from keras.utils import to_categorical

vocab_size = len(tokenizer.word_index) + 1

# separate into input and output
sequences = array(sequences)
print(sequences.shape)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]
print(y.shape)

(117290, 51)
(117290, 7324)


### 모델 훈련

In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            366200    
                                                                 
 lstm (LSTM)                 (None, 50, 100)           60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 7324)              739724    
                                                                 
Total params: 1,256,824
Trainable params: 1,256,824
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=10)

Epoch 1/10
917/917 [==============================] - 315s 334ms/step - loss: 6.1356 - accuracy: 0.0739
Epoch 2/10
917/917 [==============================] - 302s 330ms/step - loss: 5.6744 - accuracy: 0.1078
Epoch 3/10
917/917 [==============================] - 332s 362ms/step - loss: 5.4224 - accuracy: 0.1317
Epoch 4/10
917/917 [==============================] - 319s 347ms/step - loss: 5.3092 - accuracy: 0.1422
Epoch 5/10
917/917 [==============================] - 323s 352ms/step - loss: 5.1925 - accuracy: 0.1505
Epoch 6/10
917/917 [==============================] - 329s 359ms/step - loss: 5.1077 - accuracy: 0.1563
Epoch 7/10
917/917 [==============================] - 321s 350ms/step - loss: 5.0324 - accuracy: 0.1604
Epoch 8/10
917/917 [==============================] - 324s 353ms/step - loss: 4.9590 - accuracy: 0.1648
Epoch 9/10
917/917 [==============================] - 340s 371ms/step - loss: 4.8993 - accuracy: 0.1683
Epoch 10/10
917/917 [==============================] - 314s 342m

### 모델 저장

In [1]:
from pickle import dump

# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

NameError: name 'model' is not defined

## 모델 사용

In [ ]:
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

seq_length = len(lines[0].split()) - 1

In [ ]:
model = load_model('model.h5')
# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))

### 텍스트 생성